In [1]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')

import numpy as np
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time
from pprint import pprint
import copy

In [2]:
# Import for debugging in jupyter notebook
from IPython.core.debugger import set_trace #TODO remove at end.

In [3]:
libc = CDLL(find_library('c'))
cstdout = c_void_p.in_dll(libc, 'stdout')

In [4]:
class layers:
    def __init__(self):
        self.layertypes = []
        self.weights = []
        self.biases = []
        self.numlayer = 0
        self.ffn_counter = 0
        self.rank = []
        self.use_LP = []

In [5]:
def parse_bias(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    #return v.reshape((v.size,1))
    return v

In [6]:
def parse_vector(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    return v.reshape((v.size,1))
    #return v

In [7]:
def balanced_split(text):
    i = 0
    bal = 0
    start = 0
    result = []
    while i < len(text):
        if text[i] == '[':
            bal += 1
        elif text[i] == ']':
            bal -= 1
        elif text[i] == ',' and bal == 0:
            result.append(text[start:i])
            start = i+1
        i += 1
    if start < i:
        result.append(text[start:i])
    return result

In [8]:
def parse_matrix(text):
    i = 0
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    return np.array([*map(lambda x: parse_vector(x.strip()).flatten(), balanced_split(text[1:-1]))])

In [9]:
def parse_net(text):
    lines = [*filter(lambda x: len(x) != 0, text.split('\n'))]
    i = 0
    res = layers()
    while i < len(lines):
        if lines[i] in ['ReLU', 'Affine']:
            W = parse_matrix(lines[i+1])
            b = parse_bias(lines[i+2])
            res.layertypes.append(lines[i])
            res.weights.append(W)
            res.biases.append(b)
            res.numlayer+= 1
            res.rank.append(np.zeros((W.shape[0],1)))
            res.use_LP.append(np.full((W.shape[0],1), False))
            i += 3
        else:
            raise Exception('parse error: '+lines[i])
    return res

In [10]:
def parse_spec(text):
    text = text.replace("[", "")
    text = text.replace("]", "")
    with open('dummy', 'w') as my_file:
        my_file.write(text)
    data = np.genfromtxt('dummy', delimiter=',',dtype=np.double)
    low = copy.deepcopy(data[:,0])
    high = copy.deepcopy(data[:,1])
    return low,high

In [11]:
def get_perturbed_image(x, epsilon):
    image = x[1:len(x)]
    num_pixels = len(image)
    LB_N0 = image - epsilon
    UB_N0 = image + epsilon
     
    for i in range(num_pixels):
        if(LB_N0[i] < 0):
            LB_N0[i] = 0
        if(UB_N0[i] > 1):
            UB_N0[i] = 1
    return LB_N0, UB_N0

In [12]:
def generate_linexpr0(weights, bias, size):
    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_DENSE, size)
    cst = pointer(linexpr0.contents.cst)
    elina_scalar_set_double(cst.contents.val.scalar, bias)
    for i in range(size):
        elina_linexpr0_set_coeff_scalar_double(linexpr0,i,weights[i])
    return linexpr0

In [13]:
def analyze(nn, LB_N0, UB_N0, label):   
    num_pixels = len(LB_N0)
    nn.ffn_counter = 0
    numlayer = nn.numlayer 
    man = elina_box_manager_alloc()
    itv = elina_interval_array_alloc(num_pixels)
    for i in range(num_pixels):
        elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_pixels, itv)
    elina_interval_array_free(itv,num_pixels)
    for layerno in range(numlayer):
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
           weights = nn.weights[nn.ffn_counter]
           biases = nn.biases[nn.ffn_counter]
           dims = elina_abstract0_dimension(man,element)
           num_in_pixels = dims.intdim + dims.realdim
           num_out_pixels = len(weights)

           dimadd = elina_dimchange_alloc(0,num_out_pixels)    
           for i in range(num_out_pixels):
               dimadd.contents.dim[i] = num_in_pixels
           elina_abstract0_add_dimensions(man, True, element, dimadd, False)
           elina_dimchange_free(dimadd)
           np.ascontiguousarray(weights, dtype=np.double)
           np.ascontiguousarray(biases, dtype=np.double)
           var = num_in_pixels
           # handle affine layer
           for i in range(num_out_pixels):
               tdim= ElinaDim(var)
               linexpr0 = generate_linexpr0(weights[i],biases[i],num_in_pixels)
               element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
               var+=1
           dimrem = elina_dimchange_alloc(0,num_in_pixels)
           for i in range(num_in_pixels):
               dimrem.contents.dim[i] = i
           elina_abstract0_remove_dimensions(man, True, element, dimrem)
           elina_dimchange_free(dimrem)
           # handle ReLU layer 
           if(nn.layertypes[layerno]=='ReLU'):
              element = relu_box_layerwise(man,True,element,0, num_out_pixels)
           nn.ffn_counter+=1 

        else:
           print(' net type not supported')
   
    dims = elina_abstract0_dimension(man,element)
    output_size = dims.intdim + dims.realdim
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)

           
    # if epsilon is zero, try to classify else verify robustness 
    
    verified_flag = True
    predicted_label = 0
    if(LB_N0[0]==UB_N0[0]):
        for i in range(output_size):
            inf = bounds[i].contents.inf.contents.val.dbl
            flag = True
            for j in range(output_size):
                if(j!=i):
                   sup = bounds[j].contents.sup.contents.val.dbl
                   if(inf<=sup):
                      flag = False
                      break
            if(flag):
                predicted_label = i
                break    
    else:
        inf = bounds[label].contents.inf.contents.val.dbl
        for j in range(output_size):
            if(j!=label):
                sup = bounds[j].contents.sup.contents.val.dbl
                if(inf<=sup):
                    predicted_label = label
                    verified_flag = False
                    break

    elina_interval_array_free(bounds,output_size)
    elina_abstract0_free(man,element)
    elina_manager_free(man)        
    return predicted_label, verified_flag

## Define operations on abstract domain using Box approximations

In [14]:
def get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels):
    '''
    This function calculates the bounds of a ReLU operation. 
    INPUT:
        - man: pointer to elina manager
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU
    
    OUTPUT:
        - output_LB: lower bound of the outputs from ReLU layer
        - output_UB: upper bound of the outputs from ReLU layer
        - num_out_pixels: number of outputs of ReLI layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle ReLU Layer
    # ------------------------------------------------------------------
    num_out_pixels = num_in_pixels
    
    element = relu_box_layerwise(man, True, element,0, num_in_pixels)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

In [15]:
def get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose=False):
    '''
    This function calculates the bounds of a ReLU operation followed by a hidden layer. 
    INPUT:
        - man: pointer to elina manager
        - weights: weights of the hidden layer
        - biases: biases of the hidden layer
        - input_LB: lower bound of the inputs to the hidden layer
        - input_UB: upper bound of the inputs to the hidden layer
        - num_in_pixels: number of inputs to the input layer
    
    OUTPUT:
        - output_LB: lower bound of the outputs from hidden layer
        - output_UB: upper bound of the outputs from hidden layer
        - num_out_pixels: number of outputs of hidden layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle Affine Layer
    # ------------------------------------------------------------------

    # calculate number of outputs
    num_out_pixels = len(weights)
    
    if verbose:
        print("[Network] Input pixels: " + str(num_in_pixels))
        print("[Network] Shape of weights: " + str(np.shape(weights)))
        print("[Network] Shape of biases: " + str(np.shape(biases)))
        print("[Network] Out pixels: " + str(num_out_pixels))

    # Create number of neurons in the layer and populate it
    # with the number of inputs to each neuron in the layer
    dimadd = elina_dimchange_alloc(0, num_out_pixels)    
    for i in range(num_out_pixels):
        dimadd.contents.dim[i] = num_in_pixels

    # Add dimensions to an ElinaAbstract0 pointer i.e. element
    elina_abstract0_add_dimensions(man, True, element, dimadd, False)
    elina_dimchange_free(dimadd)

    # Create the linear expression associated each neuron
    var = num_in_pixels
    for i in range(num_out_pixels):
        tdim = ElinaDim(var)
        linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
        # Parallel assignment of several dimensions of an ElinaAbstract0 by using an ElinaLinexpr0Array
        element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
        var += 1

    # Pointer to which semantics we want to follow.
    dimrem = elina_dimchange_alloc(0, num_in_pixels)
    for i in range(num_in_pixels):
        dimrem.contents.dim[i] = i
        
    # Remove dimensions from an ElinaAbstract0
    elina_abstract0_remove_dimensions(man, True, element, dimrem)
    elina_dimchange_free(dimrem)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl    
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

# Define function to verify the neural network

In [16]:
def verify_network(LB_N0, UB_N0, LB_NN, UB_NN, label, num_input_pixels = 784, num_out_pixels = 10):
    '''
    This function verifies the network given the bounds of the input layer and the final layer of the network.
    INPUT:
        - LB_N0: lower bounds of the preturbed input image
        - UB_N0: unpper bounds of the preturbed input image
        - LB_NN: lower bounds of the final layer of neural network
        - UB_NN: upper bounds of the final layer of neural network
        - label: true label of the input image
        - num_input_pixels: number of pixels in the input image
        - num_out_pixels: number of neurons in the last layer of the network (for MNIST, default: 10)
    
    OUTPUT:
        - predicted_label: label predicted by the neural network
        - verified_flag: boolean variable, true if the network is robust to perturbation
    '''
    
    # if epsilon is zero, try to classify else verify robustness 
    verified_flag = True
    predicted_label = 0
    if(LB_N0[0]==UB_N0[0]):
        for i in range(num_out_pixels):
            inf = LB_NN[i]
            flag = True
            for j in range(num_out_pixels):
                if(j!=i):
                    sup = UB_NN[j]
                    if(inf<=sup):
                        flag = False
                        break
            if(flag):
                predicted_label = i
                break    
    else:
        inf = LB_NN[label]
        for j in range(num_out_pixels):
            if(j!=label):
                sup = UB_NN[j]
                if(inf<=sup):
                    predicted_label = label
                    verified_flag = False
                    break

    if(verified_flag):
        print("verified")
    else:
        print("can not be verified")  
        
    return predicted_label, verified_flag

# Initialize the problem variables

In [17]:
!ls /home/riai2018/mnist_nets/

mnist_relu_3_10.txt    mnist_relu_6_100.txt  mnist_relu_9_100.txt
mnist_relu_3_20.txt    mnist_relu_6_200.txt  mnist_relu_9_200.txt
mnist_relu_3_50.txt    mnist_relu_6_20.txt
mnist_relu_4_1024.txt  mnist_relu_6_50.txt


In [18]:
# netname = '/home/riai2018/mnist_nets/mnist_relu_3_10.txt'
# specname = '/home/riai2018/mnist_images/img2.txt'
# epsilon = 0.01072 NOT verified
# epsilon = 0.01071 verified

netname = '/home/riai2018/mnist_nets/mnist_relu_3_10.txt'
specname = '/home/riai2018/mnist_images/img2.txt'
epsilon = 0.01072

In [19]:
with open(netname, 'r') as netfile:
    netstring = netfile.read()
with open(specname, 'r') as specfile:
    specstring = specfile.read()
nn = parse_net(netstring)
x0_low, x0_high = parse_spec(specstring)
LB_N0, UB_N0 = get_perturbed_image(x0_low,0)

In [20]:
numlayer = nn.numlayer 

for layerno in range(numlayer):
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        print(nn.layertypes[layerno])

ReLU
ReLU
ReLU


# Get perturbed label (provided prediction for unperturbed is true)

In [21]:
label, _ = analyze(nn,LB_N0,UB_N0,0) # Get label of unperturbed image, i.e. eps=0
print("Test label: " + str(label))

if(label == int(x0_low[0])):
    LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
else:
    print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)

Test label: 1


# Start Verification

## 1) Find naive/spectral bounds using box approximation

In [22]:
# create a list to store the bounds found through box approximation
LB_hidden_box_list = []
UB_hidden_box_list = []

# add the bounds for the image input
LB_hidden_box_list.insert(0, LB_N0)
UB_hidden_box_list.insert(0, UB_N0)

# create manager for Elina
man = elina_box_manager_alloc()

# initialize variables for the network iteration
numlayer = nn.numlayer 
nn.ffn_counter = 0

# for input image
input_LB = LB_N0.copy()
input_UB = UB_N0.copy()
num_in_pixels = len(LB_N0)

# verbose for debugging
verbose = False

In [23]:
# NOTE: Run upper codeblock before iteration!

print("Input Layer, size: " + str(len(LB_N0)))
print('---------------')

for layerno in range(numlayer):
    
    print("Layer Number: " + str(layerno + 1))
    
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        if verbose:
            print("Layer Type: %s" % nn.layertypes[layerno])
            
        # read the layer weights and biases
        weights = nn.weights[nn.ffn_counter]
        biases = nn.biases[nn.ffn_counter]
        np.ascontiguousarray(weights, dtype=np.double)
        np.ascontiguousarray(biases, dtype=np.double)
        
        # ------------------------------------------------------------------
        # Handle Affine Layer
        # ------------------------------------------------------------------
        output_LB, output_UB, num_out_pixels = get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose)
        
        # Add bounds to the list
        LB_hidden_box_list.append(output_LB.copy())
        UB_hidden_box_list.append(output_UB.copy())
        # Prepare variables for next layer
        input_LB = output_LB.copy()
        input_UB = output_UB.copy()
        num_in_pixels = num_out_pixels
        nn.ffn_counter += 1 

        # ------------------------------------------------------------------
        # Handle ReLU Layer
        # ------------------------------------------------------------------
        if(nn.layertypes[layerno] == "ReLU"):
            output_LB, output_UB, num_out_pixels = get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels)

            # Prepare variables for next layer
            input_LB = output_LB.copy()
            input_UB = output_UB.copy()
        
        if verbose:
            print("[OUTPUT] Bounds: ")
            output_LB, output_UB  = output_LB.squeeze(), output_UB.squeeze()
            pprint(np.stack((output_LB, output_UB), axis=1))
            
        print('---------------')

    else:
        print(' net type not supported')
        
print("Output Layer, size: " + str(len(output_LB)))

elina_manager_free(man)

Input Layer, size: 784
---------------
Layer Number: 1
---------------
Layer Number: 2
---------------
Layer Number: 3
---------------
Output Layer, size: 10


In [24]:
print("[OUTPUT] Bounds: ")
pprint(np.concatenate([LB_hidden_box_list[-1], UB_hidden_box_list[-1]], axis = 1))

[OUTPUT] Bounds: 
array([[-5.64008542, -0.68057756],
       [ 3.42079412, 10.31333789],
       [-0.49786419,  3.41469333],
       [-0.22360172,  2.49013492],
       [-3.78319696, -0.98382818],
       [-3.58369875, -0.40777788],
       [-5.21258919,  0.14710902],
       [-0.86738134,  3.42412938],
       [-0.23059622,  2.84913467],
       [-2.95769429, -0.08689794]])


## 2) Find tighter bounds using linear programming

In [25]:
def relu_activations_checker(LBs, UBs):
    """
    This function computes “which side” of the ReLU the pre-ReLU activations lies on.
    INPUT
        - LBs: list of lower bound of inputs to all the layers i.e. (input, hidden_1, hidden_2, ....)
        - UBs: list of upper bound of inputs to all the layers i.e. (input, hidden_1, hidden_2, ....)
    OUTPUT:
        - neuron_states: list of neuron activation information for each layer
                         KEY:   -1: deactivated
                                +1: activated
                                 0: ambiguous  
        - alphas: slope for relu approximation when neuron state is 0
    """
    
    numlayers = len(UBs) - 1
    neuron_states = []
    alphas = []
    
    # for k layer network, we have k-1 layers of relu activation layers
    for i in range(1,  numlayers):
        # create array to store the ReLU states and slopes 
        neuron_state = np.zeros_like(UBs[i], int)
        alpha = np.zeros_like(UBs[i])
        # iterate over each pre-relu neuron activation
        for j in range(len(UBs[i])):
            u = UBs[i][j]
            l = LBs[i][j]

            if u <= 0:
                neuron_state[j] = -1
            elif l > 0:
                neuron_state[j] = 1
            else:
                neuron_state[j] = 0
                alpha[j] = u/(u - l)
        # append to the list
        neuron_states.append(neuron_state)
        alphas.append(alpha)
                                      
    return neuron_states, alphas

__Paper:__ "Provable defenses against adversarial examples via the convex outer adversarial polytope"; J. Zico Kolter, Eric Wong [[PDF]](https://machine-learning-and-security.github.io/papers/mlsec17_paper_42.pdf)

We follow the following notation: 
```
Consider a k-layer feedforward ReLU-based NN, f_θ : R^|x| → R^|y|. Given equations: 
        z_hat_{i} = W_i * z_i + b_i, where i = {1, . . . , k}
        z_i = max(z_hat_{i-1} , 0),      where i = {2, . . . , k}
and, 
        z_1 = x
        f_θ(x) ≡ z_hat_{k}
```

In [26]:
# initialize variables for the network iteration
numlayer = nn.numlayer 
nn.ffn_counter = 0
num_in_pixels = len(LB_N0)

# verbose for debugging
verbose = False
        
# neuron_states across the network
neuron_states, alphas = relu_activations_checker(LB_hidden_box_list, UB_hidden_box_list)

In [27]:
# Sanity check: for k layer network, we have k-1 layers of relu activations
for i in range(numlayer-1):
    idx_unsure = (neuron_states[i] == 0).nonzero()[0]
    print("Number of neurons with ReLU activation in layer {0}: {1} ".format(i + 1, len(neuron_states[i])))
    print("Number of neurons that are activated ambiguously: {0} ".format(len(idx_unsure)))
    print("------------------------------")

Number of neurons with ReLU activation in layer 1: 10 
Number of neurons that are activated ambiguously: 2 
------------------------------
Number of neurons with ReLU activation in layer 2: 10 
Number of neurons that are activated ambiguously: 4 
------------------------------


In [28]:
m = Model("LP")
m.setParam("outputflag", verbose)

# disable parallel Gurobi solver
m.setParam("Method", 1)  # dual simplex
m.setParam("Threads", 1) # only 1 thread

Academic license - for non-commercial use only


In [29]:
# We follow the following notations:
# z_hat_{i} = W_i * z_i + b_i, where i = {1, . . . , k}, and z_hat_{k} = y (ouput of NN)
# z_i = max(z_hat_{i-1} , 0),  where i = {2, . . . , k}, and z_1 = x (input to NN)

# for input layer:
# Variable lower bound: Note that any value less than -1e20 is treated as negative infinity. 
num_in_pixels = len(LB_N0)
img_vars = m.addVars(num_in_pixels, lb=LB_N0, ub=UB_N0, name="input_layer")

In [30]:
# for output of each ReLU
z = []
z.append(img_vars)
# for output of each hidden layer
z_hat = []

# Create variables for all layers and append to the list 
for i in range(1, numlayer + 1):
    # for layers before the final layer, z_hat and z exists
    if i < numlayer:
        # middle layer, has both z and z hat
        z_hat_hidden = m.addVars(len(UB_hidden_box_list[i]), lb=-np.inf, vtype=GRB.CONTINUOUS, name="hidden_layer_" + str(i))
        z_relu = m.addVars(len(UB_hidden_box_list[i]), vtype=GRB.CONTINUOUS, name="relu_layer_" + str(i))
        # append to the list
        z_hat.append(z_hat_hidden)
        z.append(z_relu)
    # for last layer, only z_hat exists
    else: 
        z_hat_hidden = m.addVars(len(UB_hidden_box_list[i]), lb=-np.inf, vtype=GRB.CONTINUOUS, name="output_layer") 
        # append to the list
        z_hat.append(z_hat_hidden)

m.update()

In [31]:
# Sanity check!
# Size of z should be number of relu activation layers + 1 (for input)
print("Number of relu layers: {0}".format(len(z)))
# Size of z_hat should be number of hidden layers
print("Number of hidden layers: {0}".format(len(z_hat)))
print("Size of last hidden layer: {0}".format(len(z_hat[-1])))

Number of relu layers: 3
Number of hidden layers: 3
Size of last hidden layer: 10


In [32]:
nn.ffn_counter = 0

# Adding weights constraints for k layers
for layerno in range(numlayer):
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        # read the layer weights and biases
        weights = nn.weights[nn.ffn_counter]
        biases = nn.biases[nn.ffn_counter]
        np.ascontiguousarray(weights, dtype=np.float)
        np.ascontiguousarray(biases, dtype=np.float)
        
        # output shape of the layer
        n_in = weights.shape[1]
        n_out = weights.shape[0]
        
        for i_out in range(n_out):
            constr = LinExpr() + np.asscalar(biases[i_out])
            for s in range(n_in):
                # z start from 1
                constr += z[layerno][s] * np.asscalar(weights[i_out, s])
                
            # add constraint to model
            m.addConstr(z_hat[layerno][i_out] == constr, name="hidden_constr_" + str(layerno) + "_" + str(i_out))
            
        # update counter for next iteration
        nn.ffn_counter += 1
    else:
        raise("Not a valid layer!")
        
m.update()

In [33]:
# Adding relu constraints for (k-1) layers. The loop starts from z_2 since z_1 is input
for i in range(1, numlayer):
    for j in range(len(UB_hidden_box_list[i])):
        # relu neuron is deactivated
        if (neuron_states[i-1][j] == -1):
            m.addConstr(z[i][j] == 0, \
                        name="relu_constr_deac_" + str(i) + "_" + str(j))
        # relu is purely activated
        elif (neuron_states[i-1][j] == 1):
            m.addConstr(z[i][j] == z_hat[i-1][j], \
                        name="relu_constr_acti_" + str(i) + "_" + str(j))
        # relu activation is ambiguous
        elif (neuron_states[i-1][j] == 0):
            m.addConstr(z[i][j] >= 0 , \
                         name="relu_const_ambi_pos_" + str(i) + "_" + str(j))
            m.addConstr(z[i][j] >= z_hat[i-1][j], \
                         name="relu_const_ambi_hid_" + str(i) + "_" + str(j))
            m.addConstr(z[i][j] <= np.asscalar(alphas[i-1][j])* (z_hat[i-1][j] - np.asscalar(LB_hidden_box_list[i][j])), \
                         name="relu_const_ambi_lin_" + str(i) + "_" + str(j))
        else:
            raise(RuntimeError("unknown neuron state encountered: " + str(neuron_states[i][j])))
            
m.update()

In [34]:
# storing upper and lower bounds for last layer
UB = np.empty_like(nn.biases[-1])
LB = np.empty_like(nn.biases[-1])
numlayer = nn.numlayer 

In [35]:
# Solving for each neuron in the output layer to collect bounds
# i.e. z_hat_{-1} where -1 denotes the last array in list
for i_out in range(len(UB)):            
    # Find Lower Bound
    m.setObjective(z_hat[-1][i_out], GRB.MINIMIZE)
    m.update()
    m.optimize()
    m.write("min_model_" + str(i_out) + ".lp")
    if m.status == GRB.Status.OPTIMAL:
        LB[i_out] = m.objVal
    else:
        m.computeIIS() 
        m.write('min_model.ilp')
        print('[Min] Error. Not Able to retrieve bound. Gurobi Model. Not Optimal.')
        sys.exit(0)
    # reset model 
    m.reset()

    # Find Upper Bound
    m.setObjective(z_hat[-1][i_out], GRB.MAXIMIZE)
    m.update()
    m.optimize()
    if m.status == GRB.Status.OPTIMAL:
        UB[i_out] = m.objVal
    else:
        m.write('max_model.lp')
        print('[Max] Error. Not Able to retrieve bound. Gurobi Model. Not Optimal.')
        sys.exit(0)
    # reset model 
    m.reset()

In [37]:
# check verifiability with box!
print("--------- For Box ----------")
print("[OUTPUT] Bounds: ")
pprint(np.concatenate([LB_hidden_box_list[-1], UB_hidden_box_list[-1]], axis = 1))
print("[VERIFY] Verification status: ")
flag, _ = verify_network(LB_N0, UB_N0, LB_hidden_box_list[-1], UB_hidden_box_list[-1], \
                         label, num_input_pixels = len(LB_N0), num_out_pixels = 10)

can not be verified
--------- For Box ----------
[OUTPUT] Bounds: 
array([[-5.64008542, -0.68057756],
       [ 3.42079412, 10.31333789],
       [-0.49786419,  3.41469333],
       [-0.22360172,  2.49013492],
       [-3.78319696, -0.98382818],
       [-3.58369875, -0.40777788],
       [-5.21258919,  0.14710902],
       [-0.86738134,  3.42412938],
       [-0.23059622,  2.84913467],
       [-2.95769429, -0.08689794]])
[VERIFY] Verification status: 1


In [38]:
# check verifiability with linear!
print("--------- For Linear ----------")
print("[OUTPUT] Bounds: ")
pprint(np.stack([LB, UB], axis = 1))
print("[VERIFY] Verification status: ")
flag, _ = verify_network(LB_N0, UB_N0, LB, UB, \
                         label, num_input_pixels = len(LB_N0), num_out_pixels = 10)

--------- For Linear ----------
[OUTPUT] Bounds: 
array([[-3.99136102, -2.59596921],
       [ 6.09760516,  8.15467921],
       [ 0.29984851,  2.44317553],
       [ 0.34577915,  1.73538957],
       [-2.74763824, -1.85756388],
       [-2.5840157 , -1.33043464],
       [-3.904455  , -1.04380751],
       [ 0.1000914 ,  2.5065105 ],
       [ 0.46429701,  2.0511557 ],
       [-2.26460515, -1.02078735]])
[VERIFY] Verification status: 
verified
